# Entrenamiento y predicción con BigQuery ML

## Introducción

Este tutorial muestra cómo entrenar y evaluar un modelo de clasificación binaria con BQML. También usaremos este modelo para generar predicciones.

Las dos primeras partes del tutorial explican cómo extraer los datos y prepararlos antes de entrenar su modelo.

La última parte del tutorial profundiza en el código de entrenamiento que se usa para este modelo, con un enfoque particular en los requisitos para hacerlo compatible con BigQuery ML.

## Conjunto de Datos

Este tutorial utiliza el conjunto de datos de ingresos del censo de los Estados Unidos proporcionado por el repositorio de aprendizaje automático de UC Irvine. Este conjunto de datos contiene información sobre personas de una base de datos del Censo de 1994, incluida la edad, la educación, el estado civil, la ocupación y si ganan más de $50,000 al año.

El conjunto de datos ahora existe en los conjuntos de datos públicos de BigQuery:

`bigquery-public-data.ml_datasets.census_adult_income`

## Meta

El objetivo es entrenar un modelo de clasificación binaria con BigQuery ML que prediga si una persona gana más de $50,000 al año (etiqueta de destino) en función de otra información del censo sobre la persona (características).

## Configuración del proyecto

In [1]:
# Data processing
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# BigQuery API
from google.cloud import bigquery 

In [2]:
PROJECT = 'bq-experiments-350102'

In [3]:
import os
os.environ['PROJECT'] = PROJECT

## Flujo de trabajo con un modelo de BigQuery

In this section we will build a BQML model from scratch. We will perform the following steps:
- Data preparation
- Model creation
- Model training
- Model evaluation
- Model serving

### Importaciones

In [7]:
# Procesamiento de datos
import pandas as pd

# Visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

# BigQuery API
from google.cloud import bigquery 

# Mostrar versiones de software
print(__import__('sys').version)
print(pd.__version__)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
1.3.5
